# Заполнение submit Вариант с нормированными столбцами

In [133]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

In [134]:
fl_train=pd.read_csv('HW/train.csv')
fl_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 22 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
dtypes: floa

# Обучение

In [135]:
# Удаляем id И target
train= fl_train.drop(['_id', 'target'], axis=1)
y = fl_train.target

In [136]:
# Отнормируем числовый столбцы

num_list=train.select_dtypes(include=[np.number]).columns.tolist()
for col in num_list :
    mean=train[col].mean()
    std=train[col].std()
    train[col]=train[col].map(lambda x : (x-mean)/std)

In [137]:
train=pd.get_dummies(train)

In [138]:
X_train=train
y_train=y

In [147]:
lr = LogisticRegression(penalty='l1')
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Проверка результата

In [148]:
a=30
print('Средний на кросс-валидации по {} разбиениям train-файла'.format(a))
cv = StratifiedKFold(n_splits=a, shuffle=True, random_state=15)
cvc_lr=cross_val_score(lr, X_train, y_train, scoring='roc_auc', cv=cv.split(X_train, y_train) )
print('lr', cvc_lr.mean())

Средний на кросс-валидации по 30 разбиениям train-файла
lr 0.9333405098506873


# Получаем данные на submit

In [150]:
fl_test=pd.read_csv('HW/test.csv')
fl_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 21 columns):
_id               16476 non-null object
age               16476 non-null int64
job               16476 non-null object
marital           16476 non-null object
education         16476 non-null object
default           16476 non-null object
housing           16476 non-null object
loan              16476 non-null object
contact           16476 non-null object
month             16476 non-null object
day_of_week       16476 non-null object
duration          16476 non-null int64
campaign          16476 non-null int64
pdays             16476 non-null int64
previous          16476 non-null int64
poutcome          16476 non-null object
emp.var.rate      16476 non-null float64
cons.price.idx    16476 non-null float64
cons.conf.idx     16476 non-null float64
euribor3m         16476 non-null float64
nr.employed       16476 non-null float64
dtypes: float64(5), int64(5), object(11)
memory usa

In [151]:
# Удаляем id
test= fl_test.drop(['_id'], axis=1)

In [152]:
# Отнормируем числовый столбцы

num_list=test.select_dtypes(include=[np.number]).columns.tolist()
for col in num_list :
    mean=test[col].mean()
    std=test[col].std()
    test[col]=test[col].map(lambda x : (x-mean)/std)

In [153]:
test=pd.get_dummies(test)

In [167]:
y_subm=lr.predict_proba(test)[:,1]

In [171]:
np.shape(y_subm)

(16476,)

In [124]:
# Проверим, что id в test и submit совпадают
fl_subm=pd.read_csv('HW/sample_submission.csv')
is_noteq = 0
is_eq = 0
for i in range(len(y_subm)) :
    if fl_subm._id.loc[i] != fl_test._id.loc[i] : is_noteq += 1
    if fl_subm._id.loc[i] == fl_test._id.loc[i] : is_eq += 1
is_noteq, is_eq

(0, 16476)

In [162]:
fl_subm['target'] = y_subm

In [163]:
fl_subm

,_id,target
0,66810d8e6bf2b41c880a7bc6c8a1e295,0.056419
1,ccac3879652b08cb8b44c1920fd93afa,0.009096
2,fcccab4d7a76f70647f015f2c84c2af8,0.014342
3,ed8399278c30678dab739045fa12b440,0.039577
4,1d4d62ac5cabcb48bac7112813f290cb,0.020253
5,aba2dec4c5cab88824f36babd24b986f,0.169888
6,06f318f1dd178e738f675bb88a5adb84,0.009996
7,d5036f5956e42ee6207296238fc4bc1d,0.007044
8,fe0cc8933698ad4046ff2b82f65756eb,0.016903
9,70190122f4ebf196535e11b33eb95b81,0.009769


In [164]:
fl_subm.to_csv('my_subm2.csv',index=False)